Import libraries and download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                    -failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.7.9|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='0_73_pypy|1_73_pypy|5_73_pypy|4_73_pypy|3_73_pypy|2_73_pypy']
  - geopy -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']
  - rsa -> pyt

ModuleNotFoundError: No module named 'folium'

Start venue search by colleeting lat/log of Manhattan

In [85]:
address = 'Manhattan, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7896239 -73.9598939


Search for max 50 hotels in Manhattan

In [87]:
search_query = 'hotel'
radius = 5000
print(search_query + ' .... OK!')

hotel .... OK!


Get Foursquare credentials

In [88]:
CLIENT_ID = 'KKISJGDP0Z3MXLA0QA4WLAT5OSIEQWX02AQQZ0122Q3DOCV2' # your Foursquare ID
CLIENT_SECRET = 'HZSRWMWU4SXXB1VI0J5BC1PM1B1M555H4PGZUNAMSWNLM4F4' # your Foursquare Secret
ACCESS_TOKEN = 'TTIDKPQQMX21IFNT5XICW02VFBPY3YT4CWFANP32UFOBXSJ0' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KKISJGDP0Z3MXLA0QA4WLAT5OSIEQWX02AQQZ0122Q3DOCV2
CLIENT_SECRET:HZSRWMWU4SXXB1VI0J5BC1PM1B1M555H4PGZUNAMSWNLM4F4


Define corresponding url for venue(hotel) search

In [89]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=KKISJGDP0Z3MXLA0QA4WLAT5OSIEQWX02AQQZ0122Q3DOCV2&client_secret=HZSRWMWU4SXXB1VI0J5BC1PM1B1M555H4PGZUNAMSWNLM4F4&ll=40.7896239,-73.9598939&oauth_token=TTIDKPQQMX21IFNT5XICW02VFBPY3YT4CWFANP32UFOBXSJ0&v=20180604&query=hotel&radius=5000&limit=50'

Send the GET Request and examine the results

In [90]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '601706711942f034dfc01f58'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4ad78cbff964a520140c21e3',
    'name': 'Hotel Wales',
    'location': {'address': '1295 Madison Ave',
     'crossStreet': '92nd St',
     'lat': 40.7847375,
     'lng': -73.9557131,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.7847375,
       'lng': -73.9557131}],
     'distance': 648,
     'postalCode': '10128',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['1295 Madison Ave (92nd St)', 'New York, NY 10128']},
    'categories': [{'id': '4bf58dd8d48988d1fa931735',
      'name': 'Hotel',
      'pluralName': 'Hotels',
      'shortName': 'Hotel',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1612121713',
    'hasPerk': F

Get relevant part of JSON and transform it into a pandas dataframe

In [91]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()
#dataframe['categories']

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ad78cbff964a520140c21e3,1295 Madison Ave,US,New York,United States,92nd St,648,"[1295 Madison Ave (92nd St), New York, NY 10128]","[{'label': 'display', 'lat': 40.7847375, 'lng'...",40.784737,-73.955713,NaN,10128,NY,Hotel Wales,v-1612121713,NaN
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,49d18dfdf964a5208f5b1fe3,"5th Avenue, Central Park South",US,New York,United States,at Central Park S,3049,"[5th Avenue, Central Park South (at Central Pa...","[{'label': 'display', 'lat': 40.76444602333996...",40.764446,-73.974145,NaN,10065,NY,The Plaza Hotel,v-1612121713,NaN
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b9c6ac8f964a520276736e3,215 West 94th Street,US,New York,United States,at Broadway,1106,"[215 West 94th Street (at Broadway), New York,...","[{'label': 'display', 'lat': 40.7932977, 'lng'...",40.793298,-73.972092,NaN,10025,NY,Days Inn Hotel New York City-Broadway,v-1612121713,NaN
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ad2b3d4f964a52083e220e3,45 W 81st St,US,New York,United States,at Columbus Ave,1399,"[45 W 81st St (at Columbus Ave), New York, NY ...","[{'label': 'display', 'lat': 40.78294941406415...",40.782949,-73.973964,NaN,10024,NY,Excelsior Hotel NYC,v-1612121713,90484586
4,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b1c3322f964a520210424e3,209 W 87th St,US,New York,United States,Broadway,1280,"[209 W 87th St (Broadway), New York, NY 10024]","[{'label': 'display', 'lat': 40.7889054, 'lng'...",40.788905,-73.975054,NaN,10024,NY,Belnord Hotel,v-1612121713,NaN


Define information of interest and filter dataframe

In [92]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Hotel Wales,Hotel,1295 Madison Ave,US,New York,United States,92nd St,648,"[1295 Madison Ave (92nd St), New York, NY 10128]","[{'label': 'display', 'lat': 40.7847375, 'lng'...",40.784737,-73.955713,NaN,10128,NY,4ad78cbff964a520140c21e3
1,The Plaza Hotel,Hotel,"5th Avenue, Central Park South",US,New York,United States,at Central Park S,3049,"[5th Avenue, Central Park South (at Central Pa...","[{'label': 'display', 'lat': 40.76444602333996...",40.764446,-73.974145,NaN,10065,NY,49d18dfdf964a5208f5b1fe3
2,Days Inn Hotel New York City-Broadway,Hotel,215 West 94th Street,US,New York,United States,at Broadway,1106,"[215 West 94th Street (at Broadway), New York,...","[{'label': 'display', 'lat': 40.7932977, 'lng'...",40.793298,-73.972092,NaN,10025,NY,4b9c6ac8f964a520276736e3
3,Excelsior Hotel NYC,Hotel,45 W 81st St,US,New York,United States,at Columbus Ave,1399,"[45 W 81st St (at Columbus Ave), New York, NY ...","[{'label': 'display', 'lat': 40.78294941406415...",40.782949,-73.973964,NaN,10024,NY,4ad2b3d4f964a52083e220e3
4,Belnord Hotel,Hotel,209 W 87th St,US,New York,United States,Broadway,1280,"[209 W 87th St (Broadway), New York, NY 10024]","[{'label': 'display', 'lat': 40.7889054, 'lng'...",40.788905,-73.975054,NaN,10024,NY,4b1c3322f964a520210424e3
5,Broadway Hotel,Hostel,230 W 101st St,US,New York,United States,at Broadway,1247,"[230 W 101st St (at Broadway), New York, NY 10...","[{'label': 'display', 'lat': 40.79793166906806...",40.797932,-73.969834,NaN,10025,NY,4bf2fc262d629521cbe55f58
6,Hotel 99 Llc,Hotel,244 W 99th St,US,New York,United States,NaN,1194,"[244 W 99th St, New York, NY 10025]","[{'label': 'display', 'lat': 40.79669018312864...",40.796690,-73.970555,NaN,10025,NY,4bc3a05adce4eee125af719d
7,Trump International Hotel & Tower® New York,Hotel,1 Central Park W,US,New York,United States,at 60th St,2905,"[1 Central Park W (at 60th St), New York, NY 1...","[{'label': 'display', 'lat': 40.76932505147269...",40.769325,-73.981567,Lincoln Square,10023,NY,4ac289e3f964a5209f9920e3
8,The Lucerne Hotel,Hotel,201 W 79th St,US,New York,United States,at Amsterdam Avenue,1712,"[201 W 79th St (at Amsterdam Avenue), New York...","[{'label': 'display', 'lat': 40.7834267, 'lng'...",40.783427,-73.978495,NaN,10024,NY,3fd66200f964a5204fe81ee3
9,Park Central Hotel New York,Hotel,870 7th Ave,US,New York,United States,btwn W 55th & W 56th St,3304,"[870 7th Ave (btwn W 55th & W 56th St), New Yo...","[{'label': 'display', 'lat': 40.76481, 'lng': ...",40.764810,-73.981400,NaN,10019,NY,4690ac4cf964a520a0481fe3


Visualization for all the hotels

In [93]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Manhattan',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Collect rating score for the first venue

In [94]:
venue_id = '4ad78cbff964a520140c21e3' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/4ad78cbff964a520140c21e3?client_id=KKISJGDP0Z3MXLA0QA4WLAT5OSIEQWX02AQQZ0122Q3DOCV2&client_secret=HZSRWMWU4SXXB1VI0J5BC1PM1B1M555H4PGZUNAMSWNLM4F4&oauth_token=TTIDKPQQMX21IFNT5XICW02VFBPY3YT4CWFANP32UFOBXSJ0&v=20180604'

In [95]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4ad78cbff964a520140c21e3',
 'name': 'Hotel Wales',
 'contact': {'phone': '2128766000',
  'formattedPhone': '(212) 876-6000',
  'twitter': 'hotelwalesnyc',
  'facebook': '14078676374',
  'facebookUsername': 'newyorkhotel',
  'facebookName': 'Hotel Wales New York'},
 'location': {'address': '1295 Madison Ave',
  'crossStreet': '92nd St',
  'lat': 40.7847375,
  'lng': -73.9557131,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.7847375,
    'lng': -73.9557131}],
  'postalCode': '10128',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['1295 Madison Ave (92nd St)', 'New York, NY 10128']},
 'canonicalUrl': 'https://foursquare.com/v/hotel-wales/4ad78cbff964a520140c21e3',
 'categories': [{'id': '4bf58dd8d48988d1fa931735',
   'name': 'Hotel',
   'pluralName': 'Hotels',
   'shortName': 'Hotel',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
    'suffix': '.png'},
   'primary': True}],
 'verif

In [96]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.5


We can collect rating for all the hotels through looping through venue ID as we did for first venue. Then we can make ranking based on the rating and selecte the top 10 hotels as our recommendation. Then we can do the similar exercise for other categories like museums, restaurants, shopping centers etc. and get top 10 list for each interested category. As getting rating data for each venue needs premium license while it's limited for free developer license, we just use hotel as venue example and will not proceed for other categories rating data collection. And we will just use the collected 50 hotels as example for consolidated interested total venue list and then will do segmentation/clustering based on this 50 venues. It will be the same process and algorithms if we can get consolidated venue list covering all interested categories. 

Venue segmentation by K-Means clustering using venue location(lat, log) information

In [77]:
#Remove text data(hotel names) from the dataframe and just use location data for K-Means clustering
df = dataframe_filtered[['lat', 'lng']]
#dataframe_filtered
df

,lat,lng
0,40.784737,-73.955713
1,40.764446,-73.974145
2,40.793298,-73.972092
3,40.782949,-73.973964
4,40.788905,-73.975054
5,40.797932,-73.969834
6,40.796690,-73.970555
7,40.769325,-73.981567
8,40.783427,-73.978495
9,40.764810,-73.981400


In [78]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 0, 0, 1, 1, 2, 0, 2])

In [82]:
#Insert cluster labels to each hotel and add back hotel name and venue id
df.insert(0, 'Cluster Labels', kmeans.labels_)
df['name'] = dataframe_filtered['name']
df['id'] = dataframe_filtered['id']
df

C:\Users\gong\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\gong\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Cluster Labels,lat,lng,name,id
0,1,40.784737,-73.955713,Hotel Wales,4ad78cbff964a520140c21e3
1,4,40.764446,-73.974145,The Plaza Hotel,49d18dfdf964a5208f5b1fe3
2,1,40.793298,-73.972092,Days Inn Hotel New York City-Broadway,4b9c6ac8f964a520276736e3
3,0,40.782949,-73.973964,Excelsior Hotel NYC,4ad2b3d4f964a52083e220e3
4,0,40.788905,-73.975054,Belnord Hotel,4b1c3322f964a520210424e3
5,1,40.797932,-73.969834,Broadway Hotel,4bf2fc262d629521cbe55f58
6,1,40.796690,-73.970555,Hotel 99 Llc,4bc3a05adce4eee125af719d
7,2,40.769325,-73.981567,Trump International Hotel & Tower® New York,4ac289e3f964a5209f9920e3
8,0,40.783427,-73.978495,The Lucerne Hotel,3fd66200f964a5204fe81ee3
9,2,40.764810,-73.981400,Park Central Hotel New York,4690ac4cf964a520a0481fe3


Visualize the clustered venues

In [83]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['lat'], df['lng'], df['name'], df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters